In [2]:
from __future__ import division      #除数可以显示为float

import jqdata               #导入聚宽函数库
from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import talib

import time                 #使用time stamp
import datetime             

import matplotlib.pyplot as plt

适当扩大胜负标准比例，突出特征

In [2]:
# 确定全局变量
win_num = 1.015      #获胜的比例
lose_num = 9.85      #失败的比例
even_num = 5        #等待的天数

# get_price中用到的参数，
sample_count = 2000
terminal_date = '2018-5-1'


# 最重要的数据列表，存储所有的整合数据，以便做后续的分析用
Data_Total_pd = pd.DataFrame()


确定股票列表的范围，一般挑选沪深300指数

In [3]:
#用于测试，分别注解不同的语句，控制股票序列大小
stock_list_all = ['000651.XSHE','000858.XSHE','300070.XSHE','601990.XSHG']
# stock_list_all = get_index_stocks('000300.XSHG')
# stock_list_all = ['601990.XSHG']
# print stock_list_all

生成第一列：价格

In [4]:
pd_price = pd.DataFrame()

for stock_name in set(stock_list_all):
    # 按照每个股票循环的要求，每次循环对数据进行清零
    pd_price = pd.DataFrame()
    
    # 一、开始取数据
    stock_price = get_price(stock_name, count = sample_count, end_date=terminal_date, frequency='1d', 
                            fields=['open', 'close'], fq='none')

    # 对数据进行处理，开盘、收盘价操作性太差，取中间值
    stock_price['avg'] = stock_price['open']*0.5 + stock_price['close']*0.5
  

    # 二、开始对pd_price赋值
    
    pd_price['code'] =  np.array([stock_name for i in range(0,len(list(stock_price['avg'])))])
    pd_price['date'] = np.array(stock_price.index)
    
  
    pd_price['avg_price'] =  np.array(stock_price['avg'])

    
    Data_Total_pd = Data_Total_pd.append(pd_price)

    
Data_Total_pd = Data_Total_pd.fillna(0)


生成第二列，每日的胜败关系

注意：
1、计算出指标后第二天才进行购买
2、第二天的价格计算出来的胜负才能作为胜败的依据
3、为方便处理，直接把下一天的胜负上移一行，为后续的相关性对比做准备

In [5]:
Data_Total_pd["win_rate"] = None

for stock_name in set(stock_list_all):
    
    # 记录股票的原始信息
    price_stock = []
    
    # 记录胜负信息
    win_rate_stock = []
        
    # 给单支股票的价格信息赋值
    price_stock = list(Data_Total_pd[Data_Total_pd['code'] == stock_name]['avg_price'])
    

    # 开始针对该股票的价格进行胜负判断
    # 判断标准
    # 1、大于win_num记为“win”
    # 2、小于lose_num记为“lose”
    # 3、多过even_num天记为“even”
#     print len(price_stock)
    
    # 设置序号，遍历整个价格数组
    # 注意：range函数生成的序列不包含函数中的第二个参数（也就是最后一个数）
    # 第一次遍历前半部分
    

    for i in range(0,int(len(price_stock)-even_num)):
        
        # 通过even_num天的对比，判断胜负，加入到win_rate_stock中
        for day_count in range(1,even_num+1):

            if price_stock[i+day_count] / price_stock[i] >= win_num:
                win_rate_stock.append("win")
                break
            if price_stock[i+day_count] / price_stock[i] <= lose_num:                
                win_rate_stock.append("lose")
                break
            if day_count >= even_num:
                win_rate_stock.append("even")
                break
     
    # 第二次遍历even_num后半部分 
    for i in range(int(len(price_stock)-even_num),int(len(price_stock))):
        win_rate_stock.append("even")

    
    # 将下一天的胜负上移一行，为后续的相关性对比做准备
    # 赋值时直接处理

    win_rate_stock.append("even")

    # 开始给pandas函数赋值
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"win_rate"] = np.array(win_rate_stock[1:])

Data_Total_pd = Data_Total_pd.fillna(0)


生成MACD信息

In [6]:
Data_Total_pd["MACD_dif"] = None
Data_Total_pd["MACD_dea"] = None
Data_Total_pd["MACD_macd"] = None


for stock_name in set(stock_list_all):
    dif = []
    dea = []
    macd = []
    
    macd_price = Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"avg_price"]
    

    
    dif, dea, macd = talib.MACD(np.array(macd_price), fastperiod=5, slowperiod=10, signalperiod=5)
    
#     print len(dif)
#     print len(dea)
#     print len(macd)
    
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"MACD_dif"] = np.array(dif)
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"MACD_dea"] = np.array(dea)
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"MACD_macd"] = np.array(macd)

Data_Total_pd = Data_Total_pd.fillna(0)


生成均线信息

In [7]:
Data_Total_pd["EMA_5"] = None
Data_Total_pd["EMA_10"] = None
Data_Total_pd["EMA_gap"] = None

for stock_name in set(stock_list_all):
    ema_5 = []
    ema_10 = []
    ema_gap = []
    
    
    ema_price = Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"avg_price"]
    

    ema_5 = talib.EMA(np.array(ema_price), timeperiod=5)
    ema_10 = talib.EMA(np.array(ema_price), timeperiod=10)
    ema_gap = talib.EMA(np.array(ema_price), timeperiod=5) - talib.EMA(np.array(ema_price), timeperiod=10)
    

    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"EMA_5"] = np.array(ema_5)
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"EMA_10"] = np.array(ema_10)
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"EMA_gap"] = np.array(ema_gap)
    
#     print Data_Total_pd
Data_Total_pd = Data_Total_pd.fillna(0)


生成KDJ指标

In [8]:
Data_Total_pd["KDJ_K"] = None
Data_Total_pd["KDJ_D"] = None
Data_Total_pd["KDJ_J"] = None

for stock_name in set(stock_list_all):
    K_values = []
    D_values = []
    J_values = []
    
    
    kdj_price = get_price(stock_name, count = sample_count, end_date=terminal_date, frequency='1d', 
                            fields=['high','low','close'], fq='none')
    
    
    
    kdj_price = kdj_price.fillna(0)
    
    #                  (Today's Close - LowestLow)
    # FASTK(Kperiod) = --------------------------- * 100
    #                   (HighestHigh - LowestLow)

    # FASTD(FastDperiod) = MA Smoothed FASTK over FastDperiod

    # SLOWK(SlowKperiod) = MA Smoothed FASTK over SlowKperiod

    # SLOWD(SlowDperiod) = MA Smoothed SLOWK over SlowDperiod
    
    K_values, D_values = talib.STOCH(kdj_price['high'].values,
                                   kdj_price['low'].values,
                                   kdj_price['close'].values,
                                   fastk_period=9,
                                   slowk_period=3,
                                   slowk_matype=0,
                                   slowd_period=3,
                                   slowd_matype=0)
    
    J_values = 3 * np.array(K_values) - 2 * np.array(D_values)

    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"KDJ_K"] = np.array(K_values)
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"KDJ_D"] = np.array(D_values)
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"KDJ_J"] = np.array(J_values)
    
    
#     print Data_Total_pd
    
#     print len(K_values),len(D_values),len(J_values)

#     print K_values,D_values,J_values
    
    
#     print stock_price['high'].values,stock_price['low'].values,stock_price['close'].values
#     print len(slowk), len(slowd)
#     print slowk, slowd
Data_Total_pd = Data_Total_pd.fillna(0)


开始计算RSI指标

In [9]:
Data_Total_pd["RSI"] = None

for stock_name in set(stock_list_all):
    rsi_values = []
    
    rsi_price = get_price(stock_name, count = sample_count, end_date=terminal_date, frequency='1d', 
                            fields=['high','low','close'], fq='none')
    
    rsi_price = rsi_price.fillna(0)
    rsi_values = talib.RSI(np.array(rsi_price['close']), 12)       #RSI的天数一般是6、12、24

#     print len(rsi_values)
#     print rsi_values
    
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"RSI"] = np.array(rsi_values)
    
#     print Data_Total_pd
Data_Total_pd = Data_Total_pd.fillna(0)


开始计算动量指标“MOM”

In [10]:
Data_Total_pd["MOM"] = None

# 选择一条10日均线作为中间线，判断
Data_Total_pd["MOM_gap"] = None

for stock_name in set(stock_list_all):
    MOM_values = []
    MOM_gap_values = []
    
    mom_price = list(Data_Total_pd[Data_Total_pd['code'] == stock_name]['avg_price'])
    
   
    MOM_values = talib.MOM(np.array(mom_price), timeperiod = 25)

    MOM_gap_values = talib.MOM(np.array(mom_price), timeperiod = 25) -  talib.EMA(np.array(talib.MOM(np.array(mom_price), timeperiod = 25)), timeperiod=6)
    

#     print len(MOM_values)
#     print MOM_values
    
#     print len(MOM_gap_values)
#     print MOM_gap_values
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"MOM"] = np.array(MOM_values)
    
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"MOM_gap"] = np.array(MOM_gap_values)

Data_Total_pd = Data_Total_pd.fillna(0)


开始计算能量指标
1、OBV，能量潮指标
今日OBV=昨天OBV+sgn×今天的成交量。以收盘价为例，其中，sgn是符号函数，其数值由下式决定：

sgn=+1 今日收盘价>昨日收盘价；

sgn=-1 今日收盘价<昨日收盘价；

sgn=0 今日收盘价=昨日收盘价。

2、多空比率净额= [（收盘价－最低价）－（最高价-收盘价）] ÷（ 最高价－最低价）×今天的成交量V

In [11]:
Data_Total_pd["OBV"] = None
Data_Total_pd["Long_Short_Rate_OBV"] = None

for stock_name in set(stock_list_all):
    OBV_values = []

    
    obv_price = get_price(stock_name, count = sample_count, end_date=terminal_date, frequency='1d', 
                            fields=['open','high','low','close','volume'], fq='none')
    
    obv_price['avg'] = obv_price['open']*0.5 + obv_price['close']*0.5
    obv_price = obv_price.fillna(0)
    
    OBV_values = talib.OBV(obv_price['avg'].values,double(obv_price['volume'].values))
#     print len(OBV_values)
#     print OBV_values
    
    
    # 做了归一化
    Long_Short_Rate_OBV_values = []
    
    OBV_gap = 0 
    for i in range(0,len(obv_price['close'])):
        OBV_gap = ((obv_price['close'].values[i]-obv_price['low'].values[i]) \
                   -(obv_price['high'].values[i]-obv_price['close'].values[i])) \
                  /(obv_price['high'].values[i]-obv_price['low'].values[i])

        if isnan(OBV_gap):
            OBV_gap = 0
        
        if i == 0:
            Long_Short_Rate_OBV_values.append(obv_price['volume'].values[i])
        else:
            Long_Short_Rate_OBV_values.append(float(Long_Short_Rate_OBV_values[-1]) + float(OBV_gap)*float(obv_price['volume'].values[i]))
        
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"OBV"] = np.array(OBV_values)
    Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,"Long_Short_Rate_OBV"] = np.array(Long_Short_Rate_OBV_values)       
    
Data_Total_pd = Data_Total_pd.fillna(0)


写入文件

In [12]:
write_file('Data_Total_pd1.csv', Data_Total_pd.to_csv(), append=False) #写到文件中
